# Full final script 
### (thầy/cô chỉ cần chạy cell này)

In [53]:
import pandas as pd

import os 
BASE_DIR = os.path.dirname(
    os.path.dirname(
        os.path.dirname(
            os.path.abspath('notebook/preprocessing.ipynb'))))

path18 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2018_post.csv')
path19 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2019_post.csv')
path20 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2020_post.csv')

df_18 = pd.read_csv(path18)
df_19 = pd.read_csv(path19)
df_20 = pd.read_csv(path20)

df_19.rename(columns={'EC': 'E.C', 'CO_-2 ': 'CO3', 'HCO_ - ': 'HCO3', 'Cl -': 'Cl', 'F -': 'F', 'NO3- ': 'NO3', 'SO4-2': 'SO4', 'Na+': 'Na', 'K+': 'K', 'Ca+2': 'Ca', 'Mg+2': 'Mg'}, inplace=True)
df_18.rename(columns={'NO3 ': 'NO3'}, inplace=True)
df_20.rename(columns={'NO3 ': 'NO3'}, inplace=True)

df_20 = pd.concat(
    [df_20.iloc[:, :23], 
     df_20['Classification'].replace('O.G', 'OG'), 
     df_20.iloc[:, 24:]
    ], 
    axis=1)

df = pd.concat([df_18, df_19, df_20], ignore_index=True)
df = df.drop(columns=['CO3', 'E.C', 'sno', 'district', 'mandal', 'village', 'lat_gis', 'long_gis', 'season', 'RSC  meq  / L', 'Classification.1']).copy()

gwl_mean = df['gwl'].mean()
df.fillna({'gwl': gwl_mean}, inplace=True)

df.to_csv(os.path.join(BASE_DIR, 'data/processed_data.csv'), index=False)

# Tiền xử lí

In [44]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

!mkdir data

!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2018_post.csv" -O data/ground_water_quality_2018_post.csv
!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2019_post.csv" -O data/ground_water_quality_2019_post.csv
!wget "https://raw.githubusercontent.com/Dr-Vphuc/Ground_Water_Classifier/main/data/ground_water_quality_2020_post.csv" -O data/ground_water_quality_2020_post.csv

!ls data/

In [14]:
import os 
BASE_DIR = os.path.dirname(
    os.path.dirname(
        os.path.dirname(
            os.path.abspath('notebook/preprocessing.ipynb'))))

path18 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2018_post.csv')
path19 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2019_post.csv')
path20 = os.path.join(BASE_DIR, 'data\\ground_water_quality_2020_post.csv')

In [35]:
df_18 = pd.read_csv(path18)
df_19 = pd.read_csv(path19)
df_20 = pd.read_csv(path20)

## Chuẩn hóa tên cột

In [36]:
df_19.rename(columns={'EC': 'E.C', 'CO_-2 ': 'CO3', 'HCO_ - ': 'HCO3', 'Cl -': 'Cl', 'F -': 'F', 'NO3- ': 'NO3', 'SO4-2': 'SO4', 'Na+': 'Na', 'K+': 'K', 'Ca+2': 'Ca', 'Mg+2': 'Mg'}, inplace=True)
df_18.rename(columns={'NO3 ': 'NO3'}, inplace=True)
df_20.rename(columns={'NO3 ': 'NO3'}, inplace=True)

## Chuẩn hóa giá trị

### Thống nhất class O.G năm 2020 và OG năm 2019 là OG

In [37]:
df_20 = pd.concat(
    [df_20.iloc[:, :23], 
     df_20['Classification'].replace('O.G', 'OG'), 
     df_20.iloc[:, 24:]
    ], 
    axis=1)

### Xử lí null

In [23]:
# remove null values
df_18.dropna(inplace=True)
df_19.dropna(inplace=True)
df_20.dropna(inplace=True)

In [27]:
len(df_18), len(df_19), len(df_20)

(371, 202, 365)

In [47]:
df = pd.concat([df_18, df_19, df_20], ignore_index=True)

In [ ]:
len(df[df['CO3'] == 0.0]), len(df)

(774, 938)

In [ ]:
// Sự phân chia của các giá trị CO3 khác 0 vào các class không có gì đặc biệt
// mean không quá xa trong khi đo lệch chuẩn khá lớn, không có gì đặc biệt (CELL dưới)
// Vậy drop CO3 thì sẽ dữ được 160 bản ghi mà CO3 bị null, 
// bỏ E.C vì có tương quan cao với TDS
pd.concat([
    df[df['CO3'] > 0.0]['Classification'].value_counts(),
    df.groupby('Classification').size(),
    (df[df['CO3'] > 0.0]['Classification'].value_counts() / df.groupby('Classification').size()) * 100]
    , axis = 1)  

,count,0,1
Classification,,,
C3S1,83,696,11.925287
C2S1,54,248,21.774194
C4S4,6,7,85.714286
C4S2,5,36,13.888889
C4S1,5,87,5.747126
C3S2,4,12,33.333333
C3S3,4,6,66.666667
C1S1,1,3,33.333333
C3S4,1,1,100.000000


In [ ]:
df[(df['CO3'] > 0.0) & (df['Classification'] == 'C2S1')]['CO3'].mean()
#df[(df['CO3'] > 0.0) & (df['Classification'] == 'C3S1')]['CO3'].std()

np.float64(36.371)

In [10]:
len(df[df['Classification'] == 'C2S1'])

248

In [ ]:
corr = df.drop('Classification', axis=1).corr()
plt.figure(figsize=(13,13))
sns.heatmap(corr,annot=True)
plt.show()
# bỏ E.C vì có tương quan cao với TDS